In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation,BatchNormalization 
from sklearn.metrics import mean_squared_error 
from sklearn.cross_validation import train_test_split 
import seaborn as sns 
import matplotlib.pyplot as plt 
from datetime import datetime 
from sklearn import preprocessing 

sns.set_style('darkgrid')

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def create_nn_model(input_dim, activation, layers, optimizer):
    model = Sequential()
    l_num = 0
    for l in layers:
        if l_num == 0:
            model.add(Dense(l, input_dim=input_dim, activation=activation, init='he_normal'))
        else: 
            model.add(Dense(l, activation=activation, init='he_normal')) 
        l_num = l_num + 1 
    model.compile(optimizer=optimizer, loss='mse') 
    return model

In [3]:
train=pd.read_csv('/root/Downloads/train.csv')
test=pd.read_csv('/root/Downloads/test.csv')

In [4]:
ID = 'Id' 
TARGET = 'SalePrice' 
FEATURES = train.columns.drop([ID, TARGET]) 

In [5]:
y = np.log(train[TARGET].values) 

In [6]:
train=train.drop('SalePrice',axis=1) 

In [7]:
all_date=pd.concat([train,test],axis=0)

In [8]:
f_cat = all_date[FEATURES].select_dtypes(include=['object']).columns
f_num = all_date[FEATURES].select_dtypes(exclude=['object']).columns

all_date[f_num] = all_date[f_num].fillna(all_date[f_num].mean())
all_date[f_cat] = all_date[f_cat].fillna('?')

In [9]:
dummy_cat = pd.get_dummies(all_date[f_cat])
print dummy_cat

      MSZoning_?  MSZoning_C (all)  MSZoning_FV  MSZoning_RH  MSZoning_RL  \
0              0                 0            0            0            1   
1              0                 0            0            0            1   
2              0                 0            0            0            1   
3              0                 0            0            0            1   
4              0                 0            0            0            1   
5              0                 0            0            0            1   
6              0                 0            0            0            1   
7              0                 0            0            0            1   
8              0                 0            0            0            0   
9              0                 0            0            0            1   
10             0                 0            0            0            1   
11             0                 0            0            0            1   

In [10]:
all_date = pd.DataFrame(preprocessing.scale(all_date[f_num]), columns=f_num)
print all_date

      MSSubClass  LotFrontage   LotArea  OverallQual  OverallCond  YearBuilt  \
0       0.067331    -0.202068 -0.217879     0.646183    -0.507284   1.046258   
1      -0.873616     0.501870 -0.072044    -0.063185     2.188279   0.154764   
2       0.067331    -0.061280  0.137197     0.646183    -0.507284   0.980221   
3       0.302568    -0.436714 -0.078385     0.646183    -0.507284  -1.859351   
4       0.067331     0.689587  0.518903     1.355551    -0.507284   0.947203   
5      -0.167905     0.736516  0.500515    -0.772552    -0.507284   0.716075   
6      -0.873616     0.267224 -0.010667     1.355551    -0.507284   1.079276   
7       0.067331     0.000000  0.027123     0.646183     0.391237   0.055709   
8      -0.167905    -0.859076 -0.513352     0.646183    -0.507284  -1.331059   
9       3.125411    -0.906006 -0.348496    -0.772552     0.391237  -1.066912   
10     -0.873616     0.032578  0.130856    -0.772552    -0.507284  -0.208437   
11      0.067331     0.736516  0.222669 

In [11]:
X = all_date.values
print X
print X.shape[1]

[[ 0.06733141 -0.20206753 -0.21787869 ... -0.08959196 -1.55218353
   0.15764553]
 [-0.87361603  0.50187048 -0.07204408 ... -0.08959196 -0.44692486
  -0.60296158]
 [ 0.06733141 -0.06127993  0.13719688 ... -0.08959196  1.02675338
   0.15764553]
 ...
 [-0.87361603  4.25620653  1.24680804 ... -0.08959196  1.02675338
  -1.36356868]
 [ 0.65542357 -0.34285513  0.0346054  ...  1.14431218  0.28991426
  -1.36356868]
 [ 0.06733141  0.22029527 -0.06862014 ... -0.08959196  1.7635925
  -1.36356868]]
36


In [12]:
activations = ['softplus', 'relu', 'linear']
optimizers = ['adagrad', 'rmsprop', 'adam']
layers = [[150, 1], [150, 50, 1], [200, 100, 50, 1]]
X_train, X_test, y_train, y_test = train_test_split(X[0:train.shape[0]], y, test_size=0.1, random_state=0)
nnet_params = []
nnet_score = []

In [13]:
for a in activations:
    for o in optimizers:
        for l in layers:
            model = create_nn_model(input_dim=X.shape[1], activation=a, layers=l, optimizer=o) 
            fit = model.fit(X_train, y_train, batch_size=100, nb_epoch=100, validation_split=0.1, verbose=0) 
            score = np.sqrt(model.evaluate(X_test, y_test)) 
            print("\nActivation: {} Optimizer: {} Layers: {} Score: {}\n".format(a, o, l, score)) 
            nnet_params.append(str(a) + '-' + str(o) + '-' + str(l)) 
            nnet_score.append(score)
            model.summary()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_initializer="he_normal", activation="softplus", input_dim=36)`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="he_normal", activation="softplus")`
  
/usr/local/lib/python2.7/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


146/146 [==============================] - 0s 49us/step

Activation: softplus Optimizer: adagrad Layers: [150, 1] Score: 0.557509014695

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 150)               5550      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 151       
Total params: 5,701
Trainable params: 5,701
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, kernel_initializer="he_normal", activation="softplus")`
  


146/146 [==============================] - 0s 25us/step

Activation: softplus Optimizer: adagrad Layers: [150, 50, 1] Score: 0.389007762495

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 150)               5550      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                7550      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 13,151
Trainable params: 13,151
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, kernel_initializer="he_normal", activation="softplus", input_dim=36)`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, kernel_initializer="he_normal", activation="softplus")`
  


146/146 [==============================] - 0s 31us/step

Activation: softplus Optimizer: adagrad Layers: [200, 100, 50, 1] Score: 0.313895495543

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 200)               7400      
_________________________________________________________________
dense_7 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 51        
Total params: 32,601
Trainable params: 32,601
Non-trainable params: 0
_________________________________________________________________
146/146 [==============================] - 0s 36us/step

Activation: softplus Optimizer: rmsprop Layers: [150, 1] Score: 0.2

KeyboardInterrupt: 

In [ ]:
res = pd.DataFrame({'params': nnet_params, 'score': nnet_score})
res.sort_values(['score'], ascending=True, inplace=True)
print res

In [ ]:
model = create_nn_model(input_dim=X.shape[1], activation='linear', layers=[150, 50, 1], optimizer='adagrad') 
fit = model.fit(X_train, y_train, batch_size=100, nb_epoch=100, validation_split=0.1, verbose=1)
result=model.predict(X[train.shape[0]:])
result=result.flatten()
zz=[pow(np.e,x) for x in result]

df=pd.DataFrame({ 'Id': [x for x in range(1461,2920)],
                   'SalePrice': zz })
df.to_csv('/root/Downloads/result_1.csv',index=False)